In [1]:
import os
import pandas as pd
from scipy.stats import zscore

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)

df['mpg'] = zscore(df['mpg'])
display(df)

,mpg,cylinders,displacement,...,year,origin,name
0,-0.706439,8,307.0,...,70,1,chevrolet chevelle malibu
1,-1.090751,8,350.0,...,70,1,buick skylark 320
...,...,...,...,...,...,...,...
396,0.574601,4,120.0,...,82,1,ford ranger
397,0.958913,4,119.0,...,82,1,chevy s-10


In [2]:
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)

display(df)

,id,job,area,...,retail_dense,crime,product
0,1,vv,c,...,0.492126,0.071100,b
1,2,kd,c,...,0.342520,0.400809,c
...,...,...,...,...,...,...,...
1998,1999,qp,c,...,0.598425,0.117803,c
1999,2000,pe,c,...,0.539370,0.451973,c


In [3]:
areas = list(df['area'].unique())
print(f'Number of areas: {len(areas)}')
print(f'Areas: {areas}')

Number of areas: 4
Areas: ['c', 'd', 'a', 'b']


In [4]:
dummies = pd.get_dummies(['a','b','c','d'],prefix='area')
print(dummies)

   area_a  area_b  area_c  area_d
0       1       0       0       0
1       0       1       0       0
2       0       0       1       0
3       0       0       0       1


In [5]:
dummies = pd.get_dummies(df['area'],prefix='area')
print(dummies[0:10])

    area_a  area_b  area_c  area_d
0        0       0       1       0
1        0       0       1       0
..     ...     ...     ...     ...
8        0       0       1       0
9        1       0       0       0

[10 rows x 4 columns]


In [6]:
df = pd.concat([df,dummies],axis=1)

In [7]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 10)

display(df[['id','job','area','income','area_a',
                  'area_b','area_c','area_d']])

,id,job,area,income,area_a,area_b,area_c,area_d
0,1,vv,c,50876.0,0,0,1,0
1,2,kd,c,60369.0,0,0,1,0
2,3,pe,c,55126.0,0,0,1,0
3,4,11,c,51690.0,0,0,1,0
4,5,kl,d,28347.0,0,0,0,1
...,...,...,...,...,...,...,...,...
1995,1996,vv,c,51017.0,0,0,1,0
1996,1997,kl,d,26576.0,0,0,0,1
1997,1998,kl,d,28595.0,0,0,0,1
1998,1999,qp,c,67949.0,0,0,1,0


In [8]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 5)

df.drop('area', axis=1, inplace=True)
display(df[['id','job','income','area_a',
                  'area_b','area_c','area_d']])

,id,job,income,area_a,area_b,area_c,area_d
0,1,vv,50876.0,0,0,1,0
1,2,kd,60369.0,0,0,1,0
...,...,...,...,...,...,...,...
1998,1999,qp,67949.0,0,0,1,0
1999,2000,pe,61467.0,0,0,1,0


In [9]:
import pandas as pd
import numpy as np

np.random.seed(43)
df = pd.DataFrame({
    'cont_9': np.random.rand(10)*100,
    'cat_0': ['dog'] * 5 + ['cat'] * 5,
    'cat_1': ['wolf'] * 9 + ['tiger'] * 1,
    'y': [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
})

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
display(df)

,cont_9,cat_0,cat_1,y
0,11.505457,dog,wolf,1
1,60.906654,dog,wolf,0
2,13.339096,dog,wolf,1
3,24.058962,dog,wolf,1
4,32.713906,dog,wolf,1
5,85.913749,cat,wolf,1
6,66.609021,cat,wolf,0
7,54.116221,cat,wolf,0
8,2.901382,cat,wolf,0
9,73.374830,cat,tiger,0


In [10]:
means0 = df.groupby('cat_0')['y'].mean().to_dict()
means0

{'cat': 0.2, 'dog': 0.8}

In [11]:
df['y'].mean()

0.5

In [13]:
def calc_smooth_mean(df1, df2, cat_name, target, weight):
# Compute the global mean
    mean = df[target].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(cat_name)[target].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean) / (counts + weight)

    # Replace each value by the according smoothed mean
    if df2 is None:
        return df1[cat_name].map(smooth)
    else:
        return df1[cat_name].map(smooth),df2[cat_name].map(smooth.to_dict())

In [14]:
WEIGHT = 5
df['cat_0_enc'] = calc_smooth_mean(df1=df, df2=None, 
    cat_name='cat_0', target='y', weight=WEIGHT)
df['cat_1_enc'] = calc_smooth_mean(df1=df, df2=None, 
    cat_name='cat_1', target='y', weight=WEIGHT)

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)

display(df)

,cont_9,cat_0,cat_1,y,cat_0_enc,cat_1_enc
0,11.505457,dog,wolf,1,0.65,0.535714
1,60.906654,dog,wolf,0,0.65,0.535714
2,13.339096,dog,wolf,1,0.65,0.535714
3,24.058962,dog,wolf,1,0.65,0.535714
4,32.713906,dog,wolf,1,0.65,0.535714
5,85.913749,cat,wolf,1,0.35,0.535714
6,66.609021,cat,wolf,0,0.35,0.535714
7,54.116221,cat,wolf,0,0.35,0.535714
8,2.901382,cat,wolf,0,0.35,0.535714
9,73.374830,cat,tiger,0,0.35,0.416667
